In [1]:
import os

In [2]:
%pwd

'd:\\Machine_Learning\\MLOPS\\datascience_endtoend\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Machine_Learning\\MLOPS\\datascience_endtoend'

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [7]:

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict


In [ ]:
import pandas as pd

In [10]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")

In [11]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [14]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [25]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(self,
                 config_fiePath=CONFIG_FILE_PATH,
                 params_filePath = PARAMS_FILE_PATH,
                 schema_filePath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_fiePath)
        self.params = read_yaml(params_filePath)
        self.schema = read_yaml(schema_filePath)

        create_directories([self.config.artifacts_root])
    def get_data_validation_config(self)->DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )
        return data_validation_config

In [26]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [27]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config

    def validate_columns(self)->bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_col = list(data.columns)
            all_schema = self.config.all_schema.keys()

            for col in all_col:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status:{validation_status}")
                else:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation Status:{validation_status}")
            return validation_status

        
        
        except Exception as e:
            raise e


In [28]:
try:
    config = ConfigurationManager()
    manager = config.get_data_validation_config()
    validation = DataValidation(config=manager)
    validation.validate_columns()
except Exception as e:
    raise e
    

[2025-07-28 13:17:29,238: INFO: common: yaml file <_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='UTF-8'>: loaded successfully]
[2025-07-28 13:17:29,241: INFO: common: yaml file <_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'>: loaded successfully]
[2025-07-28 13:17:29,247: INFO: common: yaml file <_io.TextIOWrapper name='schema.yaml' mode='r' encoding='UTF-8'>: loaded successfully]
[2025-07-28 13:17:29,249: INFO: common: created directory at: artifacts]
[2025-07-28 13:17:29,251: INFO: common: created directory at: artifacts/data_validation]
